In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
from sklearn.datasets import make_regression
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

def create_gui():
    def train_model():
        try:
            # Generate regression dataset
            X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            # Choose base model
            if model_choice.get() == "Decision Tree":
                base_model = DecisionTreeRegressor(random_state=42)
            else:
                base_model = LinearRegression()

            # Bagging Regressor
            bag_model = BaggingRegressor(
                estimator=base_model,
                n_estimators=int(n_estimators.get()),
                max_samples=float(samples.get()),
                bootstrap=bool(bootstrap_var.get()),
                random_state=42
            )

            # Grid Search
            param_grid = {
                'n_estimators': [50, 100, 200],
                'max_samples': [0.5, 0.7, 1.0]
            }
            grid = GridSearchCV(bag_model, param_grid, cv=5, n_jobs=-1)
            grid.fit(X_train, y_train)

            # Predictions
            y_pred = grid.best_estimator_.predict(X_test)

            # Metrics
            mse = mean_squared_error(y_test, y_pred)
            r2_train = grid.best_estimator_.score(X_train, y_train)
            r2_test = grid.best_estimator_.score(X_test, y_test)

            # Update results
            result_label.config(text=f"MSE: {mse:.4f}, Train R²: {r2_train:.3f}, Test R²: {r2_test:.3f}")
            best_params_label.config(text=f"Best Params: {grid.best_params_}")

        except Exception as e:
            messagebox.showerror("Error", str(e))

    # Main window
    window = tk.Tk()
    window.title("Bagging Regressor - Quasmi ayaan 221P001")

    # Model selection
    ttk.Label(window, text="Model:").pack(pady=5)
    model_choice = ttk.Combobox(window, values=["Decision Tree", "Linear Regression"])
    model_choice.current(0)
    model_choice.pack(pady=5)

    # Input fields
    for text, var in [("Estimators:", "100"), ("Samples (0-1):", "0.5")]:
        ttk.Label(window, text=text).pack(pady=5)
        entry = tk.Entry(window)
        entry.insert(0, var)
        entry.pack(pady=5)
        if text.startswith("Estimators"):
            n_estimators = entry
        else:
            samples = entry

    # Bootstrap checkbox
    bootstrap_var = tk.IntVar()
    tk.Checkbutton(window, text="Bootstrap", variable=bootstrap_var).pack(pady=5)

    # Train button
    tk.Button(window, text="Train Model", command=train_model, bg="#4CAF50", fg="white").pack(pady=10)

    # Result labels
    result_label = tk.Label(window, text="MSE: N/A")
    result_label.pack(pady=5)
    best_params_label = tk.Label(window, text="Best Params: N/A")
    best_params_label.pack(pady=5)

    # Run GUI
    window.mainloop()

# Run the app
create_gui()
